In [1]:
%load_ext autoreload

In [9]:
import functools
import gym
from Config import Config
from Models import ActorCritic
from Networks import cnn_head_model, actor_model, critic_model, head_model
from Memory import Memory
from baselines.common.cmd_util import make_env
from baselines.common.atari_wrappers import wrap_deepmind, make_atari

import matplotlib.pyplot as plt

config = Config(gym.make('CartPole-v1'))

config.update_every = 500
config.num_learn = 4
config.win_condition = 230
config.n_episodes = 1000
config.max_t = 700

config.Memory = Memory
config.Model = ActorCritic
config.head_model = functools.partial(head_model, config)
config.actor_model = functools.partial(actor_model, config)
config.critic_model = functools.partial(critic_model, config)


Running experiment with device: cpu


In [14]:
import copy
import gym
import torch
import numpy as np
from collections import deque
from PPO import PPOClassical
from Config import Config
import pdb

def train(config):
    env = copy.deepcopy(config.env)
    steps = 0
    scores_deque = deque(maxlen=100)
    scores = []
    average_scores = []
    max_score = -np.Inf

    agent = PPOClassical(config)

    for i_episode in range(1, config.n_episodes+1):
        state = env.reset()
        score = 0
        for t in range(config.max_t):
            steps += 1

            action, log_prob, value, entr = agent.act(state)
            next_state, reward, done, _ = env.step(action.item())

            agent.mem.add(torch.FloatTensor(state), action, reward, log_prob, done)

            # Update 
            state = next_state
            score += reward


            if steps >= config.update_every:
                agent.learn(config.num_learn)
                agent.mem.clear()
                steps = 0

            if done:
                break 

        # Book Keeping
        scores_deque.append(score)
        scores.append(score)
        average_scores.append(np.mean(scores_deque))

        if i_episode % 10 == 0:
            print("\rEpisode {}	Average Score: {:.2f}	Score: {:.2f}".format(i_episode, np.mean(scores_deque), score), end="")
        if i_episode % 100 == 0:
            print("\rEpisode {}	Average Score: {:.2f}".format(i_episode, np.mean(scores_deque)))   

        if np.mean(scores_deque) > config.win_condition:
            print("\nEnvironment Solved!")
            break

    return scores, average_scores


In [15]:
scores, average_score = train(config)
plt.plot(scores)
plt.plot(average_score)
plt.show()

AttributeError: 'numpy.ndarray' object has no attribute 'dim'